# Análise sobre os Preços dos Combustíveis no Brasil

Aluno: Rafael Pereira Cândido

Matrícula: 2221134

Curso: Ciência da Computação

Esse trabalho tem o propósito de analizar os preços dos combustíveis no Brasil do período de 2003 até 2023.

Chat sobre as queries e modelagem...: https://chatgpt.com/share/973eee34-e214-485c-8001-240b457834bf

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [28]:
url = "jdbc:postgresql://db:5432/fuel_analysis"
properties = {
    "user": "root",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

In [29]:
spark = SparkSession.builder \
    .appName("spark") \
    .master("local[*]") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.7.3.jar") \
    .getOrCreate()


In [52]:
def extract_data(path):
  df = spark.read.csv(path, sep=';', inferSchema=True, header=True)
  
  return df

def load_dim_table(table_name):
    return spark.read.jdbc(url=url, table=table_name, properties=properties)
  
def load_data_to_db(df, table_name):
    df.write.jdbc(url=url, table=table_name, mode="append", properties=properties)
    
def upsert_dim_table(df, table_name, key_columns):
        existing_df = load_dim_table(table_name)
        new_df = df.join(existing_df, key_columns, 'leftanti')
        if new_df.count() > 0:
            load_data_to_db(new_df, table_name)

In [31]:
base_path = '/home/jovyan/data/'
path = base_path+'ca-2020-02.csv'
df = extract_data(path)

In [32]:
print(df.count())

222637


In [33]:
def transform_data(df):
  # We have Values with NULL in 'Valor de Venda' column, so we need to drop them ()
  df = df.withColumn(
      "Data da Coleta",
      f.to_date(f.col("Data da Coleta").cast(StringType()), 'dd/MM/yyyy')
      )\
      .withColumn('Valor de Venda', f.regexp_replace('Valor de Venda', ',', '.'))\
      .withColumn('Valor de Venda', f.col('Valor de Venda').cast(DoubleType()))\
      .withColumn('Valor de Compra', f.regexp_replace('Valor de Compra', ',', '.'))\
      .withColumn('Valor de Compra', f.col('Valor de Compra').cast(DoubleType()))\
      .dropna(subset=['Valor de Venda'])\
      .fillna({'Complemento': 'S/C'})
      
  df = df.withColumnRenamed('Regiao - Sigla', 'Regiao_Sigla') \
       .withColumnRenamed('Estado - Sigla', 'Estado_Sigla') \
       .withColumnRenamed('CNPJ da Revenda', 'CNPJ_Revenda') \
       .withColumnRenamed('Data da Coleta', 'Data_Coleta') \
       .withColumnRenamed('Nome da Rua', 'Nome_Rua') \
       .withColumnRenamed('Numero Rua', 'Numero_Rua')\
       .withColumnRenamed('Valor de Venda', 'Valor_Venda') \
       .withColumnRenamed('Valor de Compra', 'Valor_Compra')
      
  return df

In [34]:
def create_dim_tempo(df):
    dim_tempo = df.select(
        f.col("Data_Coleta").alias("data_coleta"),
        f.dayofmonth("Data_Coleta").alias("dia"),
        f.month("Data_Coleta").alias("mes"),
        f.year("Data_Coleta").alias("ano"),
        f.dayofweek("Data_Coleta").alias("dia_semana")
    ).distinct()
    return dim_tempo

def create_dim_localizacao(df):
    dim_localizacao = df.select(
        f.col("Regiao_Sigla").alias("regiao_Sigla"), 
        f.col("Estado_Sigla").alias("estado_Sigla"), 
        f.col("Municipio").alias("municipio"), 
        f.col("Bairro").alias("bairro"), 
        f.col("Cep").alias("cep")
    ).distinct()
    return dim_localizacao

def create_dim_produto(df):
    dim_produto = df.select(f.col("Produto").alias("produto")).distinct()
    return dim_produto

def create_dim_revenda(df):
    dim_revenda = df.select(
        f.col("CNPJ_Revenda").alias("cnpj_revenda"), 
        f.col("Revenda").alias("nome_revenda")
    ).distinct()
    return dim_revenda

def create_dim_endereco(df):
    dim_endereco = df.select(
         f.col("Nome_Rua").alias("nome_rua"), 
         f.col("Numero_Rua").alias("numero_rua"), 
         f.col("Complemento").alias("complemento")
    ).distinct()
    return dim_endereco

In [35]:
df = transform_data(df)

In [36]:
print(df.count()) 

222637


In [37]:
dim_tempo = create_dim_tempo(df)
dim_localizacao = create_dim_localizacao(df)
dim_produto = create_dim_produto(df)
dim_revenda = create_dim_revenda(df)
dim_endereco = create_dim_endereco(df)

In [38]:
upsert_dim_table(dim_tempo, "dim_tempo", ["data_coleta", "dia", "mes", "ano", "dia_semana"])
upsert_dim_table(dim_localizacao, "dim_localizacao", ["regiao_sigla", "estado_sigla", "municipio", "bairro", "cep"])
upsert_dim_table(dim_produto, "dim_produto", ["produto"])
upsert_dim_table(dim_revenda, "dim_revenda", ["cnpj_revenda", "nome_revenda"])
upsert_dim_table(dim_endereco, "dim_endereco", ["nome_rua", "numero_rua", "complemento"])

In [39]:
dim_tempo_db = load_dim_table("dim_tempo")
dim_localizacao_db = load_dim_table("dim_localizacao")
dim_produto_db = load_dim_table("dim_produto")
dim_revenda_db = load_dim_table("dim_revenda")
dim_endereco_db = load_dim_table("dim_endereco")

In [40]:
def create_fato_venda(df, dim_tempo, dim_localizacao, dim_produto, dim_revenda, dim_endereco):
    fato_venda = df.join(dim_tempo, df.Data_Coleta == dim_tempo.data_coleta, 'left') \
                   .join(dim_localizacao, (df.Regiao_Sigla == dim_localizacao.regiao_sigla) & 
                                         (df.Estado_Sigla == dim_localizacao.estado_sigla) & 
                                         (df.Municipio == dim_localizacao.municipio) & 
                                         (df.Bairro == dim_localizacao.bairro) & 
                                         (df.Cep == dim_localizacao.cep), 'left') \
                   .join(dim_produto, df.Produto == dim_produto.produto, 'left') \
                   .join(dim_revenda, (df.CNPJ_Revenda == dim_revenda.cnpj_revenda) & 
                                      (df.Revenda == dim_revenda.nome_revenda), 'left') \
                   .join(dim_endereco, (df.Nome_Rua == dim_endereco.nome_rua) & 
                                       (df.Numero_Rua == dim_endereco.numero_rua) & 
                                       (df.Complemento == dim_endereco.complemento), 'left') \
                   .select(
                       "tempo_id",
                       "localizacao_id",
                       "produto_id",
                       "revenda_id",
                       "endereco_id",
                       f.col("Valor_Venda").alias('valor_venda'),
                       f.col("Valor_Compra").alias('valor_compra'),
                   )
    return fato_venda

In [41]:
fato_venda = create_fato_venda(df, dim_tempo_db, dim_localizacao_db, dim_produto_db, dim_revenda_db, dim_endereco_db)


In [43]:
load_data_to_db(fato_venda, "fato_venda")

In [47]:
resultado = aa.alias("fv") \
    .join(dim_tempo_db.alias("dt"), f.col("fv.tempo_id") == f.col("dt.tempo_id")) \
    .select(
        f.col("fv.venda_id"),
        f.col("fv.tempo_id"),
        f.col("fv.localizacao_id"),
        f.col("fv.produto_id"),
        f.col("fv.revenda_id"),
        f.col("fv.endereco_id"),
        f.col("fv.valor_venda"),
        f.col("fv.valor_compra"),
        f.col("dt.data_coleta"),
        f.col("dt.dia"),
        f.col("dt.mes"),
        f.col("dt.ano"),
        f.col("dt.dia_semana")
    )
